In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/Dataset #8/'  #change dir to your project folder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/Dataset #8/Dataset #8/'  #change dir to your project folder

In [33]:
import tensorflow as tf
from tensorflow import keras
from keras import preprocessing
from tensorflow.keras import layers ,models
import numpy as np
import os

def read_data(data_path):
    data = []
    file_names = os.listdir(data_path)
    file_names.sort(key=lambda x:int(x[:-4]))
    for file_name in file_names:
        file_path = os.path.join(data_path, file_name)
        signal_data = np.loadtxt(file_path)
        data.append(signal_data)
    data = np.array(data).transpose(0, 2, 1)
    d_shape = data.shape
    return data.reshape(d_shape[0], 1, d_shape[1], d_shape[2])


def read_label(data_path):
    data = []
    file_names = os.listdir(data_path)
    file_names.sort(key=lambda x:int(x[:-10]))
    for file_name in file_names:
        file_path = os.path.join(data_path, file_name)
        signal_data = np.loadtxt(file_path)
        data.append(signal_data)
    return np.array(data)



train_data_path = 'gdrive/My Drive/Dataset #8/Dataset #8/train/train_data'
train_label_path = 'gdrive/My Drive/Dataset #8/Dataset #8/train/train_label'
test_data_path = 'gdrive/My Drive/Dataset #8/Dataset #8/test/test_data'
test_label_path = 'gdrive/My Drive/Dataset #8/Dataset #8/test/test_label'

train_data = read_data(train_data_path).transpose(0, 2, 3, 1) # 519
train_label = read_label(train_label_path)
test_data = read_data(test_data_path).transpose(0, 2, 3, 1) # 519
test_label = read_label(test_label_path)

print(train_data.shape)
print(train_label.shape)
print(test_data.shape)
print(test_label.shape)

num_classes = len(np.unique(train_label))
num_classes

(1022, 6, 1024, 1)
(1022, 1024)
(332, 6, 1024, 1)
(332, 1024)


2

In [34]:
idx = np.random.permutation(len(train_data))
train_data = train_data[idx]
train_label = train_label[idx]

In [37]:
train_label[train_label == -1] = 0
test_label[test_label == -1] = 0

In [38]:
def make_model(input_shape):
    input_layer = keras.layers.Input(input_shape)
    
    conv = keras.layers.BatchNormalization()(input_layer)
    conv1_1 = keras.layers.Conv2D(filters=64, kernel_size=[1, 16], padding="same")(conv)
    conv1_1 = keras.layers.BatchNormalization()(conv1_1)
    conv1_1 = keras.layers.ReLU()(conv1_1)

    conv1_2 = keras.layers.Conv2D(filters=64, kernel_size=[1, 2], padding="same")(conv1_1)
    conv1_2 = keras.layers.BatchNormalization()(conv1_2)
    conv1_2 = keras.layers.ReLU()(conv1_2)
    
    conv2_1= keras.layers.MaxPooling2D(pool_size=[1, 16], strides=[1,2], padding="same")(conv1_2)
    conv2_1 = keras.layers.BatchNormalization()(conv2_1)
    
    conv2_2 = keras.layers.Conv2D(filters=128, kernel_size=[1, 16], padding="same")(conv2_1)
    conv2_2 = keras.layers.BatchNormalization()(conv2_2)
    conv2_2 = keras.layers.ReLU()(conv2_2)
    
    conv2_3 = keras.layers.Conv2D(filters=128, kernel_size=[1, 16], padding="same")(conv2_2)
    conv2_3 = keras.layers.BatchNormalization()(conv2_3)
    conv2_3 = keras.layers.ReLU()(conv2_3)
    
    conv3_1 = keras.layers.MaxPooling2D(pool_size=(1, 2), strides=[1,2], padding="same", data_format=None)(conv2_3)
    conv3_1 = keras.layers.BatchNormalization()(conv3_1)
    
    conv3_2 = keras.layers.Conv2D(filters=256, kernel_size=[1, 16], padding="same")(conv3_1)
    conv3_2 = keras.layers.BatchNormalization()(conv3_2)
    conv3_2 = keras.layers.ReLU()(conv3_2)
    
    conv3_3 = keras.layers.Conv2D(filters=256, kernel_size=[1, 16], padding="same")(conv3_2)
    conv3_3 = keras.layers.BatchNormalization()(conv3_3)
    conv3_3 = keras.layers.ReLU()(conv3_3)
    
    conv3_4 = keras.layers.Conv2D(filters=256, kernel_size=[1, 16], padding="same")(conv3_3)
    conv3_4 = keras.layers.BatchNormalization()(conv3_4)
    conv3_4 = keras.layers.ReLU()(conv3_4)
    
    conv2_4_1 = keras.layers.Conv2DTranspose(filters=128, kernel_size=[1,2],strides=[1,2], padding="same")(conv3_4)
    conv2_4 = tf.keras.layers.Concatenate(axis =3)([conv2_4_1, conv2_3])
    conv2_4 = keras.layers.BatchNormalization()(conv2_4)
    
    conv2_5 = keras.layers.Conv2D(filters=128, kernel_size=[1, 16], padding="same")(conv2_4)
    conv2_5 = keras.layers.BatchNormalization()(conv2_5)
    conv2_5 = keras.layers.ReLU()(conv2_5)
    
    conv2_6 = keras.layers.Conv2D(filters=128, kernel_size=[1, 16], padding="same")(conv2_5)
    conv2_6 = keras.layers.BatchNormalization()(conv2_6)
    conv2_6 = keras.layers.ReLU()(conv2_6)
    
    conv1_3_1 = keras.layers.Conv2DTranspose(filters=64, kernel_size=[1, 2],strides=[1,2], padding="same")(conv2_6)
    conv1_3 = tf.keras.layers.Concatenate(axis =3)([conv1_2, conv1_3_1])
    conv1_3 = keras.layers.BatchNormalization()(conv1_3)
    
    conv1_4 = keras.layers.Conv2D(filters=64, kernel_size=[1, 16], padding="same")(conv1_3)
    conv1_4 = keras.layers.BatchNormalization()(conv1_4)
    conv1_4 = keras.layers.ReLU()(conv1_4)
    
    conv1_4 = keras.layers.Conv2D(filters=64, kernel_size=[1, 16], padding="same")(conv1_4)
    conv1_4 = keras.layers.BatchNormalization()(conv1_4)
    conv1_4 = keras.layers.ReLU()(conv1_4)
    
    conv1_5 = keras.layers.Conv2D(filters=256, kernel_size=[6,1], padding="valid")(conv1_4)
    conv1_5 = keras.layers.BatchNormalization()(conv1_5)
    conv1_5 = keras.layers.ReLU()(conv1_5)
    
    #gap = keras.layers.GlobalAveragePooling2D()(conv1_5)

    output_layer = keras.layers.Conv2D(filters=1, kernel_size=[1, 1], padding="same",activation="sigmoid")(conv1_5)
    output_layer = keras.layers.BatchNormalization()(output_layer)
    out= tf.reshape(output_layer,[-1,1024])
    layers.Flatten()

    return keras.models.Model(inputs=input_layer, outputs=out)
    #out = tf.reshape(output_layer, [-1, 1024])
    #return out
     
model = make_model(input_shape=train_data.shape[1:])
keras.utils.plot_model(model, show_shapes=True)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 6, 1024, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 batch_normalization_36 (BatchN  (None, 6, 1024, 1)  4           ['input_3[0][0]']                
 ormalization)                                                                                    
                                                                                                  
 conv2d_26 (Conv2D)             (None, 6, 1024, 64)  1088        ['batch_normalization_36[0][0]'] 
                                                                                            

In [40]:
from tensorflow.keras.optimizers import SGD
opt = SGD(learning_rate=0.001)

epochs = 150
batch_size = 32
display_step = 1
data_len = len(train_data)
callbacks = [
    keras.callbacks.EarlyStopping( patience=10),
]

#model.compile(loss = "categorical_crossentropy", optimizer = 'adam' ,metrics=["accuracy"],)
optimizer = keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])
history = model.fit(
    train_data,
    train_label,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
    verbose=1,
)

Epoch 1/150
26/26 [==============================] - 29s 1s/step - loss: 3625.5422 - accuracy: 0.0845 - val_loss: 3550.8689 - val_accuracy: 0.1073
Epoch 2/150
26/26 [==============================] - 26s 1s/step - loss: 3624.3853 - accuracy: 0.0796 - val_loss: 3555.9841 - val_accuracy: 0.1415
Epoch 3/150
26/26 [==============================] - 26s 1s/step - loss: 3621.9460 - accuracy: 0.1322 - val_loss: 3541.1763 - val_accuracy: 0.2390
Epoch 4/150
26/26 [==============================] - 26s 1s/step - loss: 3619.9084 - accuracy: 0.1200 - val_loss: 3552.9609 - val_accuracy: 0.3610
Epoch 5/150
26/26 [==============================] - 26s 1s/step - loss: 3617.1267 - accuracy: 0.1248 - val_loss: 3567.6875 - val_accuracy: 0.4244
Epoch 6/150
26/26 [==============================] - 26s 1s/step - loss: 3616.1670 - accuracy: 0.1248 - val_loss: 3547.2366 - val_accuracy: 0.1707
Epoch 7/150
26/26 [==============================] - 26s 1s/step - loss: 3616.4517 - accuracy: 0.1273 - val_loss: 3535

In [41]:
#model = keras.models.load_model("best_model.h5")

test_loss, test_acc = model.evaluate(test_data, test_label)

print("Test accuracy", test_acc)
print("Test loss", test_loss)

11/11 [==============================] - 11s 172ms/step - loss: 4570.0625 - accuracy: 0.0241
Test accuracy 0.024096384644508362
Test loss 4570.0625
